In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression



In [ ]:
full_dataset = pd.read_excel("./fulldataset.xlsx", index_col=False)
full_dataset["AGE"] = full_dataset["AGE"].str.strip(" years")
full_dataset["HEIGHT"] = full_dataset["HEIGHT"].str.strip(" cm")
full_dataset["WEIGHT"] = full_dataset["WEIGHT"].str.strip(" kg")

full_dataset[["AGE", "HEIGHT", "WEIGHT", "TIME(min)", "C-Peptide", "ISR"]] = full_dataset[["AGE", "HEIGHT", "WEIGHT", "TIME(min)", "C-Peptide", "ISR"]].astype("float64")
full_dataset["BMI"] = full_dataset["WEIGHT"] / (full_dataset["HEIGHT"]/100)**2
full_dataset["BSA"] = np.sqrt((full_dataset["HEIGHT"] * full_dataset["WEIGHT"]) / 3600)
full_dataset

,sample,Sample ID,AGE,HEIGHT,SEX,SUBJECT,WEIGHT,TIME(min),C-Peptide,ISR,BMI,BSA
0,CONT1,1,53.0,163.1,F,normal,77.1,0.0,610.0,1.76,28.983208,1.868973
1,CONT1,1,53.0,163.1,F,normal,77.1,15.0,480.0,1.46,28.983208,1.868973
2,CONT1,1,53.0,163.1,F,normal,77.1,75.0,5190.0,20.30,28.983208,1.868973
3,CONT1,1,53.0,163.1,F,normal,77.1,135.0,4110.0,10.51,28.983208,1.868973
4,CONT1,1,53.0,163.1,F,normal,77.1,180.0,NaN,NaN,28.983208,1.868973
...,...,...,...,...,...,...,...,...,...,...,...,...
3301,DIAB7,176,64.0,179.4,M,NIDDM,81.1,135.0,3730.0,12.71,25.198575,2.010344
3302,DIAB7,176,64.0,179.4,M,NIDDM,81.1,255.0,3550.0,11.91,25.198575,2.010344
3303,DIAB7,176,64.0,179.4,M,NIDDM,81.1,315.0,4530.0,13.15,25.198575,2.010344
3304,DIAB7,176,64.0,179.4,M,NIDDM,81.1,375.0,4050.0,12.43,25.198575,2.010344


In [58]:
full_dataset_cleaned = full_dataset.dropna(subset=["ISR"], how="all")
full_dataset_cleaned


,sample,Sample ID,AGE,HEIGHT,SEX,SUBJECT,WEIGHT,TIME(min),C-Peptide,ISR,BMI,BSA
0,CONT1,1,53.0,163.1,F,normal,77.1,0.0,610.0,1.76,28.983208,1.868973
1,CONT1,1,53.0,163.1,F,normal,77.1,15.0,480.0,1.46,28.983208,1.868973
2,CONT1,1,53.0,163.1,F,normal,77.1,75.0,5190.0,20.30,28.983208,1.868973
3,CONT1,1,53.0,163.1,F,normal,77.1,135.0,4110.0,10.51,28.983208,1.868973
5,CONT1,1,53.0,163.1,F,normal,77.1,195.0,NaN,7.42,28.983208,1.868973
...,...,...,...,...,...,...,...,...,...,...,...,...
3301,DIAB7,176,64.0,179.4,M,NIDDM,81.1,135.0,3730.0,12.71,25.198575,2.010344
3302,DIAB7,176,64.0,179.4,M,NIDDM,81.1,255.0,3550.0,11.91,25.198575,2.010344
3303,DIAB7,176,64.0,179.4,M,NIDDM,81.1,315.0,4530.0,13.15,25.198575,2.010344
3304,DIAB7,176,64.0,179.4,M,NIDDM,81.1,375.0,4050.0,12.43,25.198575,2.010344


In [75]:
model_dataset = full_dataset_cleaned.dropna(subset=["C-Peptide"], how="all").copy()
model_dataset.drop(columns=["sample", "Sample ID"], inplace=True)
model_dataset.columns.unique()
model_dataset = pd.get_dummies(model_dataset, columns=["SEX", "SUBJECT"], drop_first=True)
model_dataset


,AGE,HEIGHT,WEIGHT,TIME(min),C-Peptide,ISR,BMI,BSA,SEX_M,SUBJECT_normal,SUBJECT_obese
0,53.0,163.1,77.1,0.0,610.0,1.76,28.983208,1.868973,False,True,False
1,53.0,163.1,77.1,15.0,480.0,1.46,28.983208,1.868973,False,True,False
2,53.0,163.1,77.1,75.0,5190.0,20.30,28.983208,1.868973,False,True,False
3,53.0,163.1,77.1,135.0,4110.0,10.51,28.983208,1.868973,False,True,False
7,53.0,163.1,77.1,255.0,3230.0,10.30,28.983208,1.868973,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...
3301,64.0,179.4,81.1,135.0,3730.0,12.71,25.198575,2.010344,True,False,False
3302,64.0,179.4,81.1,255.0,3550.0,11.91,25.198575,2.010344,True,False,False
3303,64.0,179.4,81.1,315.0,4530.0,13.15,25.198575,2.010344,True,False,False
3304,64.0,179.4,81.1,375.0,4050.0,12.43,25.198575,2.010344,True,False,False


In [77]:
correlations = model_dataset.corr()["C-Peptide"].sort_values(ascending=False)
correlations

C-Peptide         1.000000
ISR               0.778360
BMI               0.220717
WEIGHT            0.194723
BSA               0.160223
TIME(min)         0.071948
AGE              -0.009025
HEIGHT           -0.015977
SUBJECT_obese    -0.030682
SUBJECT_normal   -0.067723
SEX_M            -0.089729
Name: C-Peptide, dtype: float64